<a href="https://colab.research.google.com/github/easy1103/Statistical_Data_Analysis/blob/main/jhlee/pytorch_kobert_JH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab 을 본인의 Google drive에 접근

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## KoBERT text embedding

In [ ]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [4]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [7]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [44]:
train_data = 'insta_keywords' # keywords, fq_keywords, insta_keywords

In [45]:
if train_data == 'keywords':
  dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/Colab Notebooks/Statistical_Data_Analysis/data/keywords.tsv', field_indices=[2], num_discard_samples=1)

elif train_data == 'fq_keywords':
  dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/Colab Notebooks/Statistical_Data_Analysis/data/fq_keywords.tsv', field_indices=[2], num_discard_samples=1)

elif train_data == 'insta_keywords':
  dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/Colab Notebooks/Statistical_Data_Analysis/data/insta_keywords.tsv', field_indices=[2], num_discard_samples=1)

In [46]:
print(dataset_train)

In [47]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [48]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        # self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        # return (self.sentences[i] + (self.labels[i], ))
        return self.sentences[i]

    def __len__(self):
        return (len(self.sentences))


In [49]:
## Setting parameters
max_len = 256
batch_size = 16
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [50]:
data_train = BERTDataset(dataset_train, 0, tok, max_len, True, False)
# data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [51]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
# test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [52]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        # return self.classifier(out)
        return out

In [53]:
# model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model = BERTClassifier(bertmodel).to(device)

In [54]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [55]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [56]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [57]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [58]:
import numpy as np
import os

if train_data == 'keywords':
  save_path = '/content/drive/MyDrive/Colab Notebooks/Statistical_Data_Analysis/kobert_output/keywords'
elif train_data == 'fq_keywords':
  save_path = '/content/drive/MyDrive/Colab Notebooks/Statistical_Data_Analysis/kobert_output/fq_keywords'
elif train_data == 'insta_keywords':
  save_path = '/content/drive/MyDrive/Colab Notebooks/Statistical_Data_Analysis/kobert_output/insta_keywords'

os.makedirs(save_path, exist_ok=True)

X = np.empty((0, 768), dtype=np.float32) # it will become of kmenas trainset, should np.float32

model.eval()
for batch_id, (token_ids, valid_length, segment_ids) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    
    out = model(token_ids, valid_length, segment_ids)

    X = np.append(X, out.cpu().detach().numpy(), axis=0)

    print('{} : {}'.format(batch_id, out.shape))

np.save(os.path.join(save_path, '{}_{}.npy'.format(train_data, X.shape)), X)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/23 [00:00<?, ?it/s]

0 : torch.Size([16, 768])
1 : torch.Size([16, 768])
2 : torch.Size([16, 768])
3 : torch.Size([16, 768])
4 : torch.Size([16, 768])
5 : torch.Size([16, 768])
6 : torch.Size([16, 768])
7 : torch.Size([16, 768])
8 : torch.Size([16, 768])
9 : torch.Size([16, 768])
10 : torch.Size([16, 768])
11 : torch.Size([16, 768])
12 : torch.Size([16, 768])
13 : torch.Size([16, 768])
14 : torch.Size([16, 768])
15 : torch.Size([16, 768])
16 : torch.Size([16, 768])
17 : torch.Size([16, 768])
18 : torch.Size([16, 768])
19 : torch.Size([16, 768])
20 : torch.Size([16, 768])
21 : torch.Size([16, 768])
22 : torch.Size([13, 768])


In [35]:
print(device)

cuda:0
